<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Create-Neighborhood-Changes-Dataset" data-toc-modified-id="Create-Neighborhood-Changes-Dataset-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Create Neighborhood Changes Dataset</a></span></li><li><span><a href="#Load-Dictionary-" data-toc-modified-id="Load-Dictionary--2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load Dictionary </a></span></li><li><span><a href="#Save" data-toc-modified-id="Save-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Save</a></span></li></ul></div>

<h3>Create Neighborhood Changes Dataset

In [1]:
import pandas as pd
import numpy as np
import json
import ast
import pickle

import os
import sys
import re

module_path = os.path.abspath(os.path.join('./lib/'))

if module_path not in sys.path:   
    sys.path.append(module_path)
from utilities import *

import datetime

import matplotlib.pyplot as plt
import seaborn as sns

import os
import sys
import re
import time
import datetime

import geopandas as gpd
from geopandas.tools import geocode
from shapely.geometry import Point

%matplotlib inline

In [2]:
import datetime

def get_timestamp():
    #create timestamp for loggin and uniquely naming output files
    now = datetime.datetime.now()
    current_time_stamp = f'{now.year}'
    current_time_stamp += f'{now.year}'
    current_time_stamp += f'{now.month}'
    current_time_stamp += f'{now.day}'
    current_time_stamp += f'{now.second}'
    current_time_stamp += f'{now.microsecond}'
    return(current_time_stamp)

now = datetime.datetime.now()
this_year = current_time_stamp = f'{now.year}' 
this_year

'2019'

<h3>Load Dictionary 

The data for this dataset was constructed in the 2.2 Create New Dataset, as some of the elements were added to the Registerd Business Dataset. Here we create a new dataset where the each row represent a neighborhood and a time frame in quarters. It starts with 2001-Q1.

In [3]:
def load_dict(filename):
    with open(filename, 'rb') as st:
        source = pickle.loads(st.read())
        return source

sfd = load_dict('../tmp/sf_neighborhood_changes_dict.pkl')

In [4]:
sfd['2017-Q1']['Mission']

{'bus_opened': 283, 'bus_closed': 177, 'evictions': 31, 'med_price_sqft': 1043}

This file reads the saved neighborhood changes dictionary into a dataframe and saves it as .csv for use in EDA and modeling

In [5]:
nyears = 21
start_year = 2000
dfcols=['year', 'quarter', 'neighborhood', 
                            'bus_opened', 'bus_closed', 'evictions', 'med_price_sqft']

year = start_year
df = pd.DataFrame(columns=dfcols)
    
for i in range(1,nyears):
        
    ###For every quarter
   
    for j in range(1,5):   
        q = f'{year}-Q{j}'  
#        print (q)           
            
        ndict = sfd[q]
        ###for every neighborhood
        for n in ndict:
            mdict = sfd[q][n]
            bo = mdict['bus_opened'] 
            bc = mdict['bus_closed']
            e = mdict['evictions']
            mp = mdict['med_price_sqft']
                
            #row = [year, q, key, bo, bc, e, mp]
            row = {
                'year':year,
                'quarter':q,
                'neighborhood':n,
                'bus_opened':bo,
                'bus_closed':bc,
                'evictions':e,
                'med_price_sqft':mp
            }
            #print (row)
            dftemp = pd.DataFrame(row, index=[0])
            df = df.append(dftemp,  ignore_index=True)       
    year += 1
           



In [6]:
df.columns

Index(['year', 'quarter', 'neighborhood', 'bus_opened', 'bus_closed',
       'evictions', 'med_price_sqft'],
      dtype='object')

In [7]:
df.head()

,year,quarter,neighborhood,bus_opened,bus_closed,evictions,med_price_sqft
0,2000,2000-Q1,Dolores Heights,2,0,0,0
1,2000,2000-Q1,McLaren Park,0,0,0,0
2,2000,2000-Q1,South Beach,4,0,0,480
3,2000,2000-Q1,Forest Hill,0,0,0,364
4,2000,2000-Q1,Dogpatch,2,0,0,0


In [8]:
df[400:415]

,year,quarter,neighborhood,bus_opened,bus_closed,evictions,med_price_sqft
400,2000,2000-Q4,Alamo Square,0,0,0,0
401,2000,2000-Q4,Lone Mountain/USF,3,0,16,0
402,2000,2000-Q4,Mint Hill,0,0,0,0
403,2000,2000-Q4,Outer Sunset,0,0,0,328
404,2000,2000-Q4,Parkside,1,0,0,326
405,2000,2000-Q4,Lake Street,0,0,0,0
406,2000,2000-Q4,Silver Terrace,0,0,0,249
407,2000,2000-Q4,Glen Park,6,0,6,402
408,2000,2000-Q4,Lower Nob Hill,0,0,0,0
409,2000,2000-Q4,Sutro Heights,0,0,0,0


<h3>Save

In [10]:
#save
df.to_csv('../data/neighborhood_metrics.csv', index=False)

In [11]:
df = pd.read_csv('../data/neighborhood_metrics.csv', low_memory=False)